In [1]:
from qiskit import *

In [2]:
class QuantumLeftShift:
    def __init__(self, data_qubits, max_shifts, seed):
        self.max_shift = max_shifts
        self.seed_string = seed
        
        #to follow qiskit's little endian format
        self.seed = list(map(int, list(seed)))[::-1]
        self.n = data_qubits
        
        if len(seed) != self.n:
            raise ValueError("length of seed should be equal to number of data qubits")
    
        self.qc = QuantumCircuit(self.n + self.max_shift, self.n)
        self.shifts_done = 0
        self.__encodeSeed()
        
    def __repr__(self):
        return "QuantumShiftLeft(data_qubits = %d, max_shifts = %d, seed = \"%s\")" % (
                                                                                        self.n,
                                                                                        self.max_shift,
                                                                                        self.seed_string
                                                                                      )
    
    def __str__(self):
        print(self.qc)
        return "Data qubits: %d\nMax shifts: %d\nInitial seed: %s\nShifts performed: %d\n" % (
                                                                                              self.n, 
                                                                                              self.max_shift, 
                                                                                              self.seed_string, 
                                                                                              self.shifts_done
                                                                                             )
    
    
    def __encodeSeed(self):
        for i in range(len(self.seed)):
            if self.seed[i] == 1:
                self.qc.x(i)
    
    def constructShiftUnitary(self):
        pass
    
    

In [3]:
qsl = QuantumLeftShift(4, 3, "1011")

In [4]:
print(qsl)

     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     └───┘
q_2: ─────
     ┌───┐
q_3: ┤ X ├
     └───┘
q_4: ─────
          
q_5: ─────
          
q_6: ─────
          
c_0: ═════
          
c_1: ═════
          
c_2: ═════
          
c_3: ═════
          
Data qubits: 4
Max shifts: 3
Initial seed: 1011
Shifts performed: 0

